Hugues Fontenelle
7 October 2016

# Pima Indians Diabetes Database
## Predict the onset of diabetes based on diagnostic measures

Hi folks. I'm new to this, so let me try out what I've learned so far. Your comments are welcome!

First, let's load the data, and split it in four. It is the fold used the authors of the original paper.

In [1]:
import numpy as np

f = open("../input/diabetes.csv")
f.readline()  # skip the header
data = np.loadtxt(f, delimiter = ',')
X = data[:, :-1]
y = data[:, -1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Let's try out a bunch of classifiers, all with default parameters.

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"
        ]

classifiers = [
    KNeighborsClassifier(),
    SVC(kernel="linear"),
    SVC(kernel="rbf"),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

Now run all the classifiers, using 5-fold cross validation.

In [3]:
from sklearn.model_selection import cross_val_score

# iterate over classifiers
results = {}
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, X_train, y_train, cv=5)
    results[name] = scores

Here are the results:

In [4]:
for name, scores in results.items():
    print("%20s | Accuracy: %0.2f%% (+/- %0.2f%%)" % (name, 100*scores.mean(), 100*scores.std() * 2))

             RBF SVM | Accuracy: 64.24% (+/- 0.44%)
          Linear SVM | Accuracy: 76.04% (+/- 5.58%)
          Neural Net | Accuracy: 60.06% (+/- 23.16%)
       Decision Tree | Accuracy: 66.85% (+/- 4.62%)
    Gaussian Process | Accuracy: 68.58% (+/- 6.14%)
   Nearest Neighbors | Accuracy: 71.18% (+/- 7.56%)
                 QDA | Accuracy: 73.97% (+/- 8.84%)
            AdaBoost | Accuracy: 72.57% (+/- 8.32%)
         Naive Bayes | Accuracy: 73.62% (+/- 5.78%)
       Random Forest | Accuracy: 73.44% (+/- 3.69%)


Seems like a Linear SVM performs best.
Let's try some parameter optimization.

In [5]:
from sklearn.grid_search import GridSearchCV

clf = SVC(kernel="linear")

# prepare a range of values to test
param_grid = [
  {'C': [.01, .1, 1, 10], 'kernel': ['linear']},
 ]

grid = GridSearchCV(estimator=clf, param_grid=param_grid)
grid.fit(X_train, y_train)
print(grid)

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [0.01, 0.1, 1, 10], 'kernel': ['linear']}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)


In [6]:
# summarize the results of the grid search
print("Best score: %0.2f%%" % (100*grid.best_score_))
print("Best estimator for parameter C: %f" % (grid.best_estimator_.C))

Best score: 76.39%
Best estimator for parameter C: 0.100000


Finaly, train the Linear SVM (with param `C=0.1`) on the whole train set, and evaluate on the test set

In [7]:
clf = SVC(kernel="linear", C=0.1)
clf.fit(X_train, y_train)
y_eval = clf.predict(X_test)

In [8]:
acc = sum(y_eval == y_test) / float(len(y_test))
print("Accuracy: %.2f%%" % (100*acc))

Accuracy: 80.21%


We did it :-)

**edit**

I was _probably_ a bit lucky for this particular fold (`random_state=0`). Why would the accuracy on the test be higher than on the optimized trained set? Let's re-run a 5-fold cv on the whole data:

In [9]:
clf = SVC(kernel="linear", C=0.1)
scores_final = cross_val_score(clf, X, y, cv=5)

In [10]:
scores_final.mean(), scores_final.std()
print("Final model | Accuracy: %0.2f%% (+/- %0.2f%%)" % (100*scores_final.mean(), 100*scores_final.std() * 2))

Final model | Accuracy: 76.83% (+/- 4.31%)


..which is more realistic!

I am wondering, at which stage do I then use this test set?